# Comparison with other libraries

This notebooks extends 'evobandits_demo.ipynb' to compare the GMAB algorithm with popular alternatives.

Note that running this notebook can take very long (multiple days!) due to the number of algorithms, independent runs, and simulation cost.

Setup:

In [1]:
import json
from pathlib import Path

from evobandits import CategoricalParam, Study, GMAB
from irace import irace, Experiment, ParameterSpace, Scenario, Ordinal
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler, RandomSampler
from optuna.pruners import HyperbandPruner
from tqdm import tqdm

In [2]:
from application_example import genetic_algorithm

In [3]:
plt.style.use("default")

mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.serif"] = [
    "Computer Modern Roman",
    "Times New Roman",
    "Times",
    "DejaVu Serif",
]
mpl.rcParams["font.size"] = 14

optuna.logging.set_verbosity(optuna.logging.WARNING) # to suppress log spam

## 1. Presets for the Optimization

Identical number of runs, budget and a seed for reproduction:

In [4]:
SEED = 42
N_RUNS = 50
SIM_BUDGET = 1000
EVAL_BUDGET = 500

All variables are modeled as ordinal variables from the ranges below to ensure identical search space.

In [5]:
POP_SIZE_RANGE = [i for i in range(50, 251, 1)]
GENERATIONS_RANGE = [100, 200, 300, 400, 500]
ELITE_SPLIT_RANGE = [i * 0.01 for i in range(21)]
TOURNAMENT_SPLIT_RANGE = [i * 0.01 for i in range(11)]
CROSSOVER_RATE_RANGE = [i * 0.01 for i in range(101)]
MUTATION_RATE_RANGE = [i * 0.01 for i in range(101)]

Size of the Search space

In [ ]:
total_combinations = len(POP_SIZE_RANGE) * len(ELITE_SPLIT_RANGE) * len(GENERATIONS_RANGE) \
    * len(TOURNAMENT_SPLIT_RANGE) * len(CROSSOVER_RATE_RANGE) * len(MUTATION_RATE_RANGE)

print(f"Number of distinct solutions:\t{total_combinations:,}")

Number of distinct solutions:	2,368,213,155


## 2. Optimization

For each algorithm:
- Model the genetic_algorithm as objective function
- Model the search space
- Configure the algorithm
- Execute for preset, identical budget and runs
- Collect the results and re-sample to evaluate the "true" value of the configuration.

In [7]:
def estimate_true_value(seed, ga_configuration):
    evaluations = []
    rng = np.random.default_rng(seed)
    for _ in range(EVAL_BUDGET):
        seed_eval = rng.integers(0, 2**32 - 1, dtype=int)
        best_cost, _ = genetic_algorithm(seed=seed_eval, **ga_configuration)
        evaluations.append(best_cost)
    mean_evaluation = np.mean(evaluations)
    return mean_evaluation, evaluations

### 2.1 EvoBandits

In [8]:
def objective(seed: int, **params: dict):
    """Seeded, single-objective function to simulate the GA."""
    best_cost, _ = genetic_algorithm(seed=seed, **params)
    return best_cost

params = {
    "pop_size": CategoricalParam(POP_SIZE_RANGE),
    "generations": CategoricalParam(GENERATIONS_RANGE),
    "elite_split": CategoricalParam(ELITE_SPLIT_RANGE), 
    "tournament_split": CategoricalParam(TOURNAMENT_SPLIT_RANGE),
    "mutation_rate": CategoricalParam(MUTATION_RATE_RANGE), 
    "crossover_rate": CategoricalParam(CROSSOVER_RATE_RANGE), 
}

print("\nRunning optimization ...")
results_evobandits = []

for run_id in tqdm(range(N_RUNS), desc="EvoBandits | Run"):
    seed = SEED + run_id
    gmab = GMAB(population_size=10)
    study = Study(seed=seed, algorithm=gmab)
    study.optimize(objective, params, n_trials=SIM_BUDGET)
    print(f"Config:\t{study.best_params}")
    print(f"Value:\t{study.best_value}")

    mean_evaluation, evaluations = estimate_true_value(seed, study.best_params)
    print(f"Est. true value:\t{mean_evaluation}")

    results_evobandits.append({
        "mean_evaluation": mean_evaluation,
        "evaluations": evaluations,
        "best_solution": study.best_solution,
        "seed": seed
    })

    with open(Path("_data/04_results_evobandits.json"), 'w') as f:
        json.dump(results_evobandits, f)


Running optimization ...


EvoBandits | Run:   0%|          | 0/50 [00:00<?, ?it/s]

Config:	{'pop_size': 226, 'generations': 500, 'elite_split': 0.04, 'tournament_split': 0.05, 'mutation_rate': 0.78, 'crossover_rate': 0.0}
Value:	7.925576596461831


EvoBandits | Run:   2%|▏         | 1/50 [1:31:11<74:28:22, 5471.48s/it]

Est. true value:	8.009099365950854
Config:	{'pop_size': 250, 'generations': 500, 'elite_split': 0.08, 'tournament_split': 0.04, 'mutation_rate': 0.84, 'crossover_rate': 0.0}
Value:	7.866737136573285


...

Est. true value:	8.059633556200282
Config:	{'pop_size': 250, 'generations': 500, 'elite_split': 0.03, 'tournament_split': 0.07, 'mutation_rate': 0.84, 'crossover_rate': 0.0}
Value:	7.8601310346791795


EvoBandits | Run:  98%|█████████▊| 49/50 [70:27:49<1:24:52, 5092.49s/it]

Est. true value:	7.984837240667651
Config:	{'pop_size': 234, 'generations': 500, 'elite_split': 0.03, 'tournament_split': 0.08, 'mutation_rate': 0.8, 'crossover_rate': 0.13}
Value:	7.903063362310793


EvoBandits | Run: 100%|██████████| 50/50 [71:51:50<00:00, 5174.22s/it]  

Est. true value:	7.997906602060502


### 2.2 IRACE

In [12]:
def ga_runner(experiment: Experiment, scenario: Scenario):
    """Seeded, single-objective function to simulate the GA with irace."""
    best_cost, _ = genetic_algorithm(
        seed=experiment.seed, **experiment.configuration
    )
    return float(best_cost)

param_space = ParameterSpace(
    [
        Ordinal("pop_size", POP_SIZE_RANGE),
        Ordinal("generations", GENERATIONS_RANGE),
        Ordinal("elite_split", ELITE_SPLIT_RANGE),
        Ordinal("tournament_split", TOURNAMENT_SPLIT_RANGE),
        Ordinal("crossover_rate", CROSSOVER_RATE_RANGE),
        Ordinal("mutation_rate", MUTATION_RATE_RANGE),
    ]
)

print("\nRunning optimization ...")
results_irace = []

for run_id in tqdm(range(N_RUNS), desc="IRACE | Run"):
    print("\nRunning optimization ...")
    seed = SEED + run_id
    scenario = Scenario(
        max_experiments=SIM_BUDGET,
        verbose=0,
    )
    best_params = irace(ga_runner, param_space, scenario)[0]
    print(f"Config:\t{best_params}")

    mean_evaluation, evaluations = estimate_true_value(seed, best_params)
    print(f"Est. true value:\t{mean_evaluation}")

    results_irace.append({
        "mean_evaluation": mean_evaluation,
        "evaluations": evaluations,
        "best_params": best_params,
        "seed": seed,
    })

    with open(Path("_data/04_results_irace.json"), 'w') as f:
        json.dump(results_irace, f)


Running optimization ...


IRACE | Run:   0%|          | 0/50 [00:00<?, ?it/s]


Running optimization ...


Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.
c:\Users\fwuer\Code\evobandits-demo\.venv\Lib\site-packages\rpy2\rinterface\__init__.py:1211: UserWarning: Environment variable "PATH" redefined by R and overriding existing variable. Current: "c:\Users\fwuer\Code\evobandits-demo\.venv\Scripts;C:\Program Files (x86)\Intel\iCLS Client\;C:\Program Files\Intel\iCLS Client\;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files\Intel\WiFi\bin\;C:\Program Files\Common Files\Intel\WirelessCommon\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Git\cmd;C:\Users\fwuer\AppData\Local\Programs\Python\Launc

Config:	OrderedDict({'pop_size': 199, 'generations': 500, 'elite_split': 0.11, 'tournament_split': 0.09, 'crossover_rate': 0.08, 'mutation_rate': 0.96})


...

Est. true value:	8.023503910994682

Running optimization ...
Config:	OrderedDict({'pop_size': 234, 'generations': 500, 'elite_split': 0.16, 'tournament_split': 0.07, 'crossover_rate': 0.09, 'mutation_rate': 0.79})


IRACE | Run:  98%|█████████▊| 49/50 [51:53:55<1:07:28, 4048.94s/it]

Est. true value:	8.000981283021197

Running optimization ...
Config:	OrderedDict({'pop_size': 228, 'generations': 500, 'elite_split': 0.1, 'tournament_split': 0.09, 'crossover_rate': 0.01, 'mutation_rate': 0.66})


IRACE | Run: 100%|██████████| 50/50 [53:01:51<00:00, 3818.23s/it]  

Est. true value:	8.010732107198164


### 2.3 Hyperband - TPESampler

In [ ]:
# rng to make objective reproducible
MAXINT = 2 ** 32 - 1
rng = np.random.default_rng(SEED)

def objective(trial):
    """Seeded, single-objective function to simulate the GA."""
    seed = rng.integers(0, MAXINT, dtype=int)
    best_cost, _ = genetic_algorithm(
        pop_size=trial.suggest_categorical("pop_size", POP_SIZE_RANGE),
        generations=trial.suggest_categorical("generations", GENERATIONS_RANGE),
        elite_split=trial.suggest_categorical("elite_split", ELITE_SPLIT_RANGE),
        tournament_split=trial.suggest_categorical("tournament_split", TOURNAMENT_SPLIT_RANGE),
        mutation_rate=trial.suggest_categorical("mutation_rate", MUTATION_RATE_RANGE),
        crossover_rate=trial.suggest_categorical("crossover_rate", CROSSOVER_RATE_RANGE),
        seed=seed
    )

    trial.report(best_cost, 1)
    if trial.should_prune():
        raise optuna.TrialPruned()
    
    return best_cost

print("\nRunning optimization ...")
results_hyperband_tpe = []

for run_id in tqdm(range(N_RUNS), desc="Hyperband | Run"):
    seed = SEED + run_id
    gmab = GMAB(population_size=10)
    sampler = TPESampler(seed=seed) # sampling 
    pruner = HyperbandPruner() # deterministic, takes no seed

    study = optuna.create_study(sampler=sampler, pruner=pruner)
    study.optimize(objective, n_trials=SIM_BUDGET)
    print(f"Config:\t{study.best_params}")
    print(f"Value:\t{study.best_trials}")

    mean_evaluation, evaluations = estimate_true_value(seed, study.best_params)
    print(f"Est. true value:\t{mean_evaluation}")

    results_hyperband_tpe.append({
        "mean_evaluation": mean_evaluation,
        "evaluations": evaluations,
        "best_solution": study.best_params,
        "seed": seed
    })

    with open(Path("_data/04_results_hyperband_tpe.json"), 'w') as f:
        json.dump(results_hyperband_tpe, f)

### 2.5 Hyperband

In [ ]:
# rng to make objective reproducible
MAXINT = 2 ** 32 - 1
rng = np.random.default_rng(SEED)

def objective(trial):
    """Seeded, single-objective function to simulate the GA."""
    seed = rng.integers(0, MAXINT, dtype=int)
    best_cost, _ = genetic_algorithm(
        pop_size=trial.suggest_categorical("pop_size", POP_SIZE_RANGE),
        generations=trial.suggest_categorical("generations", GENERATIONS_RANGE),
        elite_split=trial.suggest_categorical("elite_split", ELITE_SPLIT_RANGE),
        tournament_split=trial.suggest_categorical("tournament_split", TOURNAMENT_SPLIT_RANGE),
        mutation_rate=trial.suggest_categorical("mutation_rate", MUTATION_RATE_RANGE),
        crossover_rate=trial.suggest_categorical("crossover_rate", CROSSOVER_RATE_RANGE),
        seed=seed
    )

    trial.report(best_cost, 1)
    if trial.should_prune():
        raise optuna.TrialPruned()
    
    return best_cost

print("\nRunning optimization ...")
results_hyperband_rs = []

for run_id in tqdm(range(N_RUNS), desc="Hyperband | Run"):
    seed = SEED + run_id
    gmab = GMAB(population_size=10)
    sampler = RandomSampler(seed=seed) # sampling 
    pruner = HyperbandPruner() # deterministic, takes no seed

    study = optuna.create_study(sampler=sampler, pruner=pruner)
    study.optimize(objective, n_trials=SIM_BUDGET)
    print(f"Config:\t{study.best_params}")
    print(f"Value:\t{study.best_trials}")

    mean_evaluation, evaluations = estimate_true_value(seed, study.best_params)
    print(f"Est. true value:\t{mean_evaluation}")

    results_hyperband_rs.append({
        "mean_evaluation": mean_evaluation,
        "evaluations": evaluations,
        "best_solution": study.best_params,
        "seed": seed
    })

    with open(Path("_data/04_results_hyperband_rs.json"), 'w') as f:
        json.dump(results_hyperband_rs, f)

### 2.5 Random Search / 1 Simulation per Configuration

In [ ]:
# rng to make objective reproducible
MAXINT = 2 ** 32 - 1
rng = np.random.default_rng(SEED)

def objective(trial):
    """Seeded, single-objective function to simulate the GA."""
    seed = rng.integers(0, MAXINT, dtype=int)
    best_cost, _ = genetic_algorithm(
        pop_size=trial.suggest_categorical("pop_size", POP_SIZE_RANGE),
        generations=trial.suggest_categorical("generations", GENERATIONS_RANGE),
        elite_split=trial.suggest_categorical("elite_split", ELITE_SPLIT_RANGE),
        tournament_split=trial.suggest_categorical("tournament_split", TOURNAMENT_SPLIT_RANGE),
        mutation_rate=trial.suggest_categorical("mutation_rate", MUTATION_RATE_RANGE),
        crossover_rate=trial.suggest_categorical("crossover_rate", CROSSOVER_RATE_RANGE),
        seed=seed
    )
    return best_cost

print("\nRunning optimization ...")
results_rs_one= []

for run_id in tqdm(range(N_RUNS), desc="Hyperband | Run"):
    seed = SEED + run_id
    gmab = GMAB(population_size=10)
    sampler = RandomSampler(seed=seed) # sampling 

    study = optuna.create_study(sampler=sampler)
    study.optimize(objective, n_trials=SIM_BUDGET)
    print(f"Config:\t{study.best_params}")
    print(f"Value:\t{study.best_trials}")

    mean_evaluation, evaluations = estimate_true_value(seed, study.best_params)
    print(f"Est. true value:\t{mean_evaluation}")

    results_rs_one.append({
        "mean_evaluation": mean_evaluation,
        "evaluations": evaluations,
        "best_solution": study.best_params,
        "seed": seed
    })

    with open(Path("_data/04_results_rs_one.json"), 'w') as f:
        json.dump(results_rs_one, f)

### 2.5 Random Search / 10 Simulations per Configuration

In [ ]:
# Random Search with Static Sampling
SIM_PER_TRIAL = 10

# rng to make objective reproducible
MAXINT = 2 ** 32 - 1
rng = np.random.default_rng(SEED)

def objective(trial):
    """Seeded, single-objective function to simulate the GA."""
    seed = rng.integers(0, MAXINT, dtype=int)
    config = {
        "pop_size": trial.suggest_categorical("pop_size", POP_SIZE_RANGE),
        "generations": trial.suggest_categorical("generations", GENERATIONS_RANGE),
        "elite_split": trial.suggest_categorical("elite_split", ELITE_SPLIT_RANGE),
        "tournament_split": trial.suggest_categorical("tournament_split", TOURNAMENT_SPLIT_RANGE),
        "mutation_rate": trial.suggest_categorical("mutation_rate", MUTATION_RATE_RANGE),
        "crossover_rate": trial.suggest_categorical("crossover_rate", CROSSOVER_RATE_RANGE),
    }

    samples = []
    for i in range(SIM_PER_TRIAL):
        best_cost, _ = genetic_algorithm(seed=seed, **config)
        samples.append(best_cost)

    return np.mean(samples)

print("\nRunning optimization ...")
results_rs_ten= []

for run_id in tqdm(range(N_RUNS), desc="Hyperband | Run"):
    seed = SEED + run_id
    gmab = GMAB(population_size=10)
    sampler = RandomSampler(seed=seed) # sampling 

    study = optuna.create_study(sampler=sampler)
    study.optimize(objective, n_trials=SIM_BUDGET / SIM_PER_TRIAL)
    print(f"Config:\t{study.best_params}")
    print(f"Value:\t{study.best_trials}")

    mean_evaluation, evaluations = estimate_true_value(seed, study.best_params)
    print(f"Est. true value:\t{mean_evaluation}")

    results_rs_ten.append({
        "mean_evaluation": mean_evaluation,
        "evaluations": evaluations,
        "best_solution": study.best_params,
        "seed": seed
    })

    with open(Path("_data/04_results_rs_ten.json"), 'w') as f:
        json.dump(results_rs_ten, f)

## 3. Comparison

Loads pre-computed results (if available)

In [ ]:
with open("_data/04_results_evobandits.json", "r") as f:
    results_evobandits = json.load(f)

In [ ]:
with open("_data/04_results_irace.json", "r") as f:
    results_irace = json.load(f)

In [ ]:
with open("_data/04_results_hyperband_tpe.json", "r") as f:
    results_hyperband_tpe = json.load(f)

In [ ]:
with open("_data/04_results_hyperband_rs.json", "r") as f:
    results_hyperband_rs = json.load(f)

In [ ]:
with open("_data/04_results_rs_one.json", "r") as f:
    results_rs_one = json.load(f)

In [ ]:
with open("_data/04_results_rs_ten.json", "r") as f:
    results_rs_ten = json.load(f)

### 3.1 Stats

In [ ]:
mean_evaluation_data = {
    "EvoBandits": [r["mean_evaluation"] for r in results_evobandits],
    "IRACE": [r["mean_evaluation"] for r in results_irace],
    "Hyperband": [r["mean_evaluation"] for r in results_hyperband_rs],
    f"Hyperband\n+TPE": [r["mean_evaluation"] for r in results_hyperband_tpe],
    "RS1": [r["mean_evaluation"] for r in results_rs_one],
    "RS10": [r["mean_evaluation"] for r in results_rs_one],
}

In [ ]:
df = pd.DataFrame({
    'value': np.concatenate(list(mean_evaluation_data.values())),
    'method': np.repeat(list(mean_evaluation_data.keys()), [len(v) for v in mean_evaluation_data.values()])
})

summary = df.groupby('method').agg(
    mean=('value', 'mean'),
    std=('value', 'std'),
    count=('value', 'size')
).reset_index()

summary['sem'] = summary['std'] / np.sqrt(summary['count'])
summary['ci95_low'] = summary['mean'] - 1.96 * summary['sem']
summary['ci95_high'] = summary['mean'] + 1.96 * summary['sem']

print(summary[['method', 'mean', 'ci95_low', 'ci95_high']])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.boxplot(
    mean_evaluation_data.values(), 
    tick_labels=mean_evaluation_data.keys(), 
    showmeans=True, 
    meanprops={"markerfacecolor":"black", "markeredgecolor":"black"}, 
    medianprops={"color": "C0"}
)
ax.set_ylabel("Total Distance")

plt.tight_layout()
plt.savefig(Path("_plots/06_comparison.pdf"))
plt.show()